In [ ]:
import time
import random
import json
from datetime import datetime
from google.cloud import pubsub_v1

project_id = 'vadimzaripov-477-2022062208552'
topic_id = 'timeseries-pos'

publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project_id, topic_id)

# Generate a single sample data record
def generate_record():
    store_id = random.randint(1, 10)
    timestamp = datetime.utcnow().isoformat() #UTC datetime
    pos_number = random.randint(1, 5)
    product_id = random.randint(1, 20)
    value = round(random.uniform(10.0, 100.0), 2)
    return {
        'store_id': store_id,
        'timestamp': timestamp,
        'pos_number': pos_number,
        'product_id': product_id,
        'value': value
    }

# Publish a batch of 50 messages to Pub/Sub
def publish_batch_to_pubsub():
    messages = [generate_record() for _ in range(50)]
    for data in messages:
        data_str = json.dumps(data)  # Serialize to JSON
        #print(f"Sending JSON to Pub/Sub: {data_str}")
        data_bytes = data_str.encode('utf-8')
        future = publisher.publish(topic_path, data_bytes)
        #print(f"Published message ID {future.result()}")

# Run the function every minute
def run_periodically():

    total_sent = 0
    while True:
        publish_batch_to_pubsub()
        print("Batch of 50 messages sent. Waiting for a minute...")
        total_sent += 50
        print(f"Total sent in this session is {total_sent}")
        time.sleep(60)  # Wait for a minute

# Run the function
try:
    run_periodically()
except KeyboardInterrupt:
    print("Stopped by user")
